שחר תורג'מן - 316223304
עדן שמואל - 209348697

Libraries

In [1]:
!pip install selenium

In [2]:
!pip install webdriver_manager

In [3]:
!pip install beautifulsoup4

For some reason, I need the code below to make Selenium work on my computer:

In [15]:
import sys
sys.path.append(r'c:\users\shaha\appdata\roaming\python\python312\site-packages')

Selenium imports for web browser automation

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Other necessary imports

In [17]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import csv
import datetime

### MAIN FUNCTIONS AND CODE FOR ONE PAGE WEB SCRAPER

In [18]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('window-size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
main_driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)


Function to get the number of images per ad on the specified page:

In [19]:
def image_num(num, main_driver):
    main_driver.get(f'https://www.ad.co.il/car?sp261=13910&pageindex={num}')
    soup = BeautifulSoup(main_driver.page_source,'html.parser')
    card_blocks=soup.find_all('div',class_='card-block')
    lst = []
    x = 1
    for card_block in card_blocks:
            if x < 49:
                data_images = card_block.get('data-images')
                lst.append(data_images)
                x += 1
            else:
                break
    return lst

Functions to access the website, click every single ad for KIA cars, and obtain all the needed data:

In [20]:

def ad_car_webscraper_page(num):
    number_of_ads_per_page = 48
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('window-size=1920x1080')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    ad_car_driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)
    ad_car_driver.get(f'https://www.ad.co.il/car?sp261=13910&pageindex={num}')
    soup = BeautifulSoup(ad_car_driver.page_source,'html.parser')
    templist = []
    for n in range(1,number_of_ads_per_page+1):
        try:
            ads = WebDriverWait(ad_car_driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, f'/html/body/main/div[2]/div[3]/div[3]/div[2]/div[3]/div[{n}]')))
        except:
            break
        image_number = int(image_num(num, main_driver)[n-1])
        for ad in ads:
            
            ad.click()
            try:
                table_info = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div[2]/div[1]/table')))
                title_price = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="product-container"]/div[2]/div[1]/div/div[1] ')))[0]
                date = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div[2]/div[2]/div[2]/div[3]/div[1]')))
                description = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div[2]/div[2]/div[2]/div[1]')))
            except:
                try:
                    description = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div/div[2]/div/div[1]')))
                    table_info = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div/div[1]/table')))
                    title_price = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="product-container"]/div/div[1]/div/div[1]')))[0]
                    date = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div[2]/div[2]/div[2]/div[2]/div')))
                    
                except:
                    try:
                        date = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div/div[2]/div/div[3]/div')))
                    except:
                        try:
                            date = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div/div[2]/div/div[3]/div')))
                        except:
                            date = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div/div[2]/div/div[2]/div')))
                        
            table_info = table_info.text
            title_price = title_price.text
            date = date.text
            ad_number = f'{n}/48'
            print(f' page : {num} || post number : {ad_number}')
            table_dict = {'title_price':title_price,'table_info':table_info,'date':date,'image_number':image_number,'description':description.text}
            templist.append(table_dict)
            close = WebDriverWait(ad_car_driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pop-modal"]/div/div/div[1]/div/button')))
            close.click() 
    ad_car_driver.quit()
    return templist


Execute the web scraper for the KIA cars. For now, we have seven pages of ads and are getting the data from the website.

In [21]:
full_data_list = []
for i in range(1,8):
    full_data_list.extend(ad_car_webscraper_page(i))
full_data=pd.DataFrame(full_data_list)

 page : 1 || post number : 1/48
 page : 1 || post number : 2/48
 page : 1 || post number : 3/48
 page : 1 || post number : 4/48
 page : 1 || post number : 5/48
 page : 1 || post number : 6/48
 page : 1 || post number : 7/48
 page : 1 || post number : 8/48
 page : 1 || post number : 9/48
 page : 1 || post number : 10/48
 page : 1 || post number : 11/48
 page : 1 || post number : 12/48
 page : 1 || post number : 13/48
 page : 1 || post number : 14/48
 page : 1 || post number : 15/48
 page : 1 || post number : 16/48
 page : 1 || post number : 17/48
 page : 1 || post number : 18/48
 page : 1 || post number : 19/48
 page : 1 || post number : 20/48
 page : 1 || post number : 21/48
 page : 1 || post number : 22/48
 page : 1 || post number : 23/48
 page : 1 || post number : 24/48
 page : 1 || post number : 25/48
 page : 1 || post number : 26/48
 page : 1 || post number : 27/48
 page : 1 || post number : 28/48
 page : 1 || post number : 29/48
 page : 1 || post number : 30/48
 page : 1 || post n

In [22]:
full_data

,title_price,table_info,date,image_number,description
0,"קיה פורטה\n16,000 ₪","שנה 2011\nיד 2\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 26/05/2024\nתאריך הקפצה אחרון: 07...,5,"לרציניים בלבד .קיה פורטה, הרכב היום בבעלות פרט..."
1,"קיה פיקנטו\n28,500 ₪","שנה 2016\nיד 1\nת. הילוכים אוטומטית\nנפח 1,200...",תאריך יצירה: 31/05/2024\nתאריך הקפצה אחרון: 31...,3,מצב מצוין לא עברה שום תיקון צבע או תאונה כל שהיא
2,"קיה פיקנטו\n15,000 ₪","שנה 2014\nיד 3\nת. הילוכים ידנית\nנפח 1,000\nס...",תאריך יצירה: 21/04/2024\nתאריך הקפצה אחרון: 18...,1,קטנה שמורה עם טסט לשנה
3,"קיה ספיה\n86,000 ₪","שנה 2019\nיד 2\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 01/05/2024\nתאריך הקפצה אחרון: 17...,5,"קיה ספורטאז טורבו דיזל, חסכונית בדלק, שמורה, מ..."
4,"קיה ריו\n18,000 ₪","שנה 2012\nיד 4\nת. הילוכים ידנית\nנפח 1,400\nס...",תאריך יצירה: 07/05/2024\nתאריך הקפצה אחרון: 07...,6,רכב חסכוני בטירוף אחד ל 22 טסט לשנה אחרי טיפול...
...,...,...,...,...,...
285,"קיה קרניבל\n110,000 ₪","שנה 2018\nיד 1\nת. הילוכים אוטומטית\nנפח 2,000...",תאריך יצירה: 01/06/2022\nתאריך הקפצה אחרון: 01...,0,ללא תאונות דגם ספורטאג ולא כפי שמופיע בתחילת ה...
286,"קיה סראטו\n12,500 ₪","שנה 2008\nיד 2\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 25/05/2022\nתאריך הקפצה אחרון: 25...,0,רכב שמור מנוע חזק
287,,"שנה 2016\nיד 1\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 16/11/2021\nתאריך הקפצה אחרון: 16...,1,
288,"קיה ספיה\n75,000 ₪","שנה 2016\nיד 1\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 09/05/2022\nתאריך הקפצה אחרון: 09...,0,"גיפון קיה ספורטג מציאה,שמורה מאוד, טאס עד 2023..."


Arranging the data:

In [23]:
full_data[['manufactor_model','price']] = full_data['title_price'].str.split('\n', n = 1, expand = True)
full_data[['manufactor','model']] = full_data['manufactor_model'].str.split(' ', n = 1, expand = True)

In [24]:
split_data = full_data['table_info'].str.split('\n', expand=True)
full_data['year'] = split_data.apply(lambda row: next((str(value).replace('שנה', '').strip()  for value in row if 'שנה' in str(value)), pd.NA),axis = 1)
full_data['hand'] = split_data.apply(lambda row: next((str(value).replace('יד', '').strip()  for value in row if 'יד' in str(value)), pd.NA),axis = 1)
full_data['gear'] = split_data.apply(lambda row: next((str(value).replace('ת. הילוכים', '').strip() for value in row if 'ת. הילוכים' in str(value)), pd.NA),axis = 1)
full_data['capacity_engine'] = split_data.apply(lambda row: next((str(value).replace('נפח', '').strip() for value in row if 'נפח' in str(value)), pd.NA),axis = 1)
full_data['engine_type'] = split_data.apply(lambda row: next((str(value).replace('סוג מנוע', '').strip() for value in row if 'סוג מנוע' in str(value)), pd.NA),axis = 1)
full_data['km'] = split_data.apply(lambda row: next((str(value).replace('ק"מ', '').strip() for value in row if 'ק"מ' in str(value)), pd.NA),axis = 1)
full_data['test'] = split_data.apply(lambda row: next((str(value).replace('טסט עד', '').strip() for value in row if 'טסט' in str(value)), pd.NA),axis = 1)
full_data['prev_ownership'] = split_data.apply(lambda row: next((str(value).replace('בעלות קודמת', '').strip() for value in row if 'בעלות קודמת' in str(value)), pd.NA),axis = 1)
full_data['curr_ownership'] = split_data.apply(lambda row: next((str(value).replace('בעלות נוכחית', '').strip() for value in row if 'בעלות נוכחית' in str(value)), pd.NA),axis = 1)
full_data['area'] = split_data.apply(lambda row: next((str(value).replace('אזור', '').strip() for value in row if 'אזור' in str(value)), pd.NA),axis = 1)
full_data['city'] = split_data.apply(lambda row: next((str(value).replace('עיר', '').strip() for value in row if 'עיר' in str(value)), pd.NA),axis = 1)
full_data['color'] = split_data.apply(lambda row: next((str(value).replace('צבע', '').strip() for value in row if 'צבע' in str(value)), pd.NA),axis = 1)
full_data['pic_num'] = full_data['image_number']
full_data[['cre_date','repub_date']] = full_data['date'].str.split('\n', n = 1, expand = True)
full_data['cre_date'] = full_data['cre_date'].str.replace('תאריך יצירה:','')
full_data['repub_date'] = full_data['repub_date'].str.replace('תאריך הקפצה אחרון:','')
full_data['description'] = full_data['description'].str.split('\n').str.join(' ')
full_data['price'] = full_data['price'].str.replace('₪','')
full_data['model'] = full_data['model'].fillna(pd.NA)
full_data['manufactor'] = full_data['manufactor'].fillna(pd.NA)

In [25]:
full_data

,title_price,table_info,date,image_number,description,manufactor_model,price,manufactor,model,year,...,km,test,prev_ownership,curr_ownership,area,city,color,pic_num,cre_date,repub_date
0,"קיה פורטה\n16,000 ₪","שנה 2011\nיד 2\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 26/05/2024\nתאריך הקפצה אחרון: 07...,5,"לרציניים בלבד .קיה פורטה, הרכב היום בבעלות פרט...",קיה פורטה,"16,000",קיה,פורטה,2011,...,"227,000",12/2024,השכרה,פרטית,מושבים בשפלה,פתחיה,כסוף מטאלי,5,26/05/2024,07/06/2024
1,"קיה פיקנטו\n28,500 ₪","שנה 2016\nיד 1\nת. הילוכים אוטומטית\nנפח 1,200...",תאריך יצירה: 31/05/2024\nתאריך הקפצה אחרון: 31...,3,מצב מצוין לא עברה שום תיקון צבע או תאונה כל שהיא,קיה פיקנטו,"28,500",קיה,פיקנטו,2016,...,"170,000",02/2025,פרטית,פרטית,רמת גן - גבעתיים,בני ברק,אפור,3,31/05/2024,31/05/2024
2,"קיה פיקנטו\n15,000 ₪","שנה 2014\nיד 3\nת. הילוכים ידנית\nנפח 1,000\nס...",תאריך יצירה: 21/04/2024\nתאריך הקפצה אחרון: 18...,1,קטנה שמורה עם טסט לשנה,קיה פיקנטו,"15,000",קיה,פיקנטו,2014,...,"135,000",03/2025,פרטית,פרטית,עמק יזרעאל,גן נר,ורוד,1,21/04/2024,18/05/2024
3,"קיה ספיה\n86,000 ₪","שנה 2019\nיד 2\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 01/05/2024\nתאריך הקפצה אחרון: 17...,5,"קיה ספורטאז טורבו דיזל, חסכונית בדלק, שמורה, מ...",קיה ספיה,"86,000",קיה,ספיה,2019,...,"95,000",08/2024,ליסינג,פרטית,עכו - נהריה,נהריה,אפור עכבר,5,01/05/2024,17/05/2024
4,"קיה ריו\n18,000 ₪","שנה 2012\nיד 4\nת. הילוכים ידנית\nנפח 1,400\nס...",תאריך יצירה: 07/05/2024\nתאריך הקפצה אחרון: 07...,6,רכב חסכוני בטירוף אחד ל 22 טסט לשנה אחרי טיפול...,קיה ריו,"18,000",קיה,ריו,2012,...,"207,000",04/2025,פרטית,פרטית,אשדוד - אשקלון,אשדוד,לבן שנהב,6,07/05/2024,07/05/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,"קיה קרניבל\n110,000 ₪","שנה 2018\nיד 1\nת. הילוכים אוטומטית\nנפח 2,000...",תאריך יצירה: 01/06/2022\nתאריך הקפצה אחרון: 01...,0,ללא תאונות דגם ספורטאג ולא כפי שמופיע בתחילת ה...,קיה קרניבל,"110,000",קיה,קרניבל,2018,...,"68,500",<NA>,<NA>,<NA>,כרמיאל והסביבה,כרמיאל,<NA>,0,01/06/2022,01/06/2022
286,"קיה סראטו\n12,500 ₪","שנה 2008\nיד 2\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 25/05/2022\nתאריך הקפצה אחרון: 25...,0,רכב שמור מנוע חזק,קיה סראטו,"12,500",קיה,סראטו,2008,...,"170,000",<NA>,<NA>,<NA>,ירושלים והסביבה,ירושלים,<NA>,0,25/05/2022,25/05/2022
287,,"שנה 2016\nיד 1\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 16/11/2021\nתאריך הקפצה אחרון: 16...,1,,,None,,<NA>,2016,...,"204,000",<NA>,פרטית,פרטית,<NA>,יערה,טורקיז,1,16/11/2021,16/11/2021
288,"קיה ספיה\n75,000 ₪","שנה 2016\nיד 1\nת. הילוכים אוטומטית\nנפח 1,600...",תאריך יצירה: 09/05/2022\nתאריך הקפצה אחרון: 09...,0,"גיפון קיה ספורטג מציאה,שמורה מאוד, טאס עד 2023...",קיה ספיה,"75,000",קיה,ספיה,2016,...,"60,000",<NA>,<NA>,<NA>,מושבים בשרון,תנובות,<NA>,0,09/05/2022,09/05/2022


In [26]:
full_data['price'] = full_data['price'].str.replace(',','',regex = True)
full_data['km'] = full_data['km'].str.replace(',','',regex = True)
full_data['capacity_engine'] = full_data['capacity_engine'].str.replace(',','',regex = True)

Reordering columns and renaming specific columns in DataFrame

In [27]:
column_names = ['manufactor','year','model','hand', 'gear', 'capacity_engine', 'engine_type','prev_ownership', 'curr_ownership','area', 'city','price','pic_num','cre_date','repub_date','description', 'color', 'km', 'test' ]
full_data = full_data[column_names]

In [28]:
full_data = full_data.rename(columns = {'year': 'Year', 'hand': 'Hand', 'gear': 'Gear', 'capacity_engine': 'capacity_Engine', 'engine_type': 'Engine_type', 'prev_ownership': 'Prev_ownership', 'curr_ownership': 'Curr_ownership', 'area': 'Area',  
                                      'city': 'City', 'price': 'Price', 'pic_num': 'Pic_num', 'cre_date': 'Cre_date', 'repub_date': 'Repub_date', 'description': 'Description', 'color': 'Color', 'km': 'Km', 'test': 'Test'})

In [29]:
full_data

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test
0,קיה,2011,פורטה,2,אוטומטית,1600,בנזין,השכרה,פרטית,מושבים בשפלה,פתחיה,16000,5,26/05/2024,07/06/2024,"לרציניים בלבד .קיה פורטה, הרכב היום בבעלות פרט...",כסוף מטאלי,227000,12/2024
1,קיה,2016,פיקנטו,1,אוטומטית,1200,בנזין,פרטית,פרטית,רמת גן - גבעתיים,בני ברק,28500,3,31/05/2024,31/05/2024,מצב מצוין לא עברה שום תיקון צבע או תאונה כל שהיא,אפור,170000,02/2025
2,קיה,2014,פיקנטו,3,ידנית,1000,בנזין,פרטית,פרטית,עמק יזרעאל,גן נר,15000,1,21/04/2024,18/05/2024,קטנה שמורה עם טסט לשנה,ורוד,135000,03/2025
3,קיה,2019,ספיה,2,אוטומטית,1600,דיזל,ליסינג,פרטית,עכו - נהריה,נהריה,86000,5,01/05/2024,17/05/2024,"קיה ספורטאז טורבו דיזל, חסכונית בדלק, שמורה, מ...",אפור עכבר,95000,08/2024
4,קיה,2012,ריו,4,ידנית,1400,דיזל,פרטית,פרטית,אשדוד - אשקלון,אשדוד,18000,6,07/05/2024,07/05/2024,רכב חסכוני בטירוף אחד ל 22 טסט לשנה אחרי טיפול...,לבן שנהב,207000,04/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,קיה,2018,קרניבל,1,אוטומטית,2000,בנזין,<NA>,<NA>,כרמיאל והסביבה,כרמיאל,110000,0,01/06/2022,01/06/2022,ללא תאונות דגם ספורטאג ולא כפי שמופיע בתחילת ה...,<NA>,68500,<NA>
286,קיה,2008,סראטו,2,אוטומטית,1600,בנזין,<NA>,<NA>,ירושלים והסביבה,ירושלים,12500,0,25/05/2022,25/05/2022,רכב שמור מנוע חזק,<NA>,170000,<NA>
287,,2016,<NA>,1,אוטומטית,1600,היבריד,פרטית,פרטית,<NA>,יערה,None,1,16/11/2021,16/11/2021,,טורקיז,204000,<NA>
288,קיה,2016,ספיה,1,אוטומטית,1600,בנזין,<NA>,<NA>,מושבים בשרון,תנובות,75000,0,09/05/2022,09/05/2022,"גיפון קיה ספורטג מציאה,שמורה מאוד, טאס עד 2023...",<NA>,60000,<NA>


Converting the types:

In [30]:
full_data.dtypes

manufactor         object
Year               object
model              object
Hand               object
Gear               object
capacity_Engine    object
Engine_type        object
Prev_ownership     object
Curr_ownership     object
Area               object
City               object
Price              object
Pic_num             int64
Cre_date           object
Repub_date         object
Description        object
Color              object
Km                 object
Test               object
dtype: object

In [31]:
columns_to_convert_int = ['Year', 'Hand', 'capacity_Engine', 'Km', 'Pic_num']
full_data[columns_to_convert_int] = full_data[columns_to_convert_int].apply(pd.to_numeric, errors='coerce')
for column in columns_to_convert_int:
    full_data[column] = full_data[column].astype('Int64')

In [32]:
columns_to_convert_category = ['Gear', 'Engine_type', 'Prev_ownership', 'Curr_ownership']
for column in columns_to_convert_category:
    full_data[column] = full_data[column].astype('category')

In [33]:
columns_to_convert_str = ['Area', 'City', 'Description', 'Color']
for column in columns_to_convert_str:
    full_data[column] = full_data[column].astype('string')

In [34]:
full_data['Price'] = full_data['Price'].astype(float)

In [35]:
full_data['Cre_date']=pd.to_datetime(full_data['Cre_date'],dayfirst=True)
full_data['Repub_date']=pd.to_datetime(full_data['Repub_date'],dayfirst=True)

#We were required to search for data only on cars in the years 1980-2016

In [77]:
filtered_data = full_data[(full_data['Year'] >= 1980) & (full_data['Year'] <= 2016)]

Preprocessing and calculating days until test dates

In [78]:
from datetime import datetime

def preprocess_date_string(date_str):
    if pd.isna(date_str):
        return np.nan
        
    if '/' in date_str:
        parts = date_str.split('/')
    elif '-' in date_str:
        parts = date_str.split('-')
    else:
        return np.nan
    
    if len(parts) == 2:  
        full_months = ["01", "03", "05", "07", "08", "10", "12"]
        month, year  = parts
        if month in full_months:
            day = '31' 
        elif month == "02":
            day = '28'
        else:
            day = '30'
                
    elif len(parts) == 3:  
        day, month, year = parts
    else:
        return np.nan
    
    # Ensure all parts have leading zeros if necessary
    year = year.zfill(2)
    month = month.zfill(2)
    day = day.zfill(2)
    
    # Assume any two-digit year 'YY' represents '20YY'
    if len(year) == 2:
        year = '20' + year
    
    return f'{year}-{month}-{day}'

filtered_data.loc[:, 'Test'] = filtered_data['Test'].apply(preprocess_date_string)
filtered_data.loc[:, 'Test'] = pd.to_datetime(filtered_data['Test'], errors='coerce')

In [79]:
def days_until(date):
    today = datetime.today()
    delta = date - today
    return delta.days

filtered_data['Test'] = filtered_data['Test'].apply(days_until)

In [80]:
filtered_data['Test'] = filtered_data['Test'].astype('Int64')

In [81]:
filtered_data.dtypes

manufactor                 object
Year                        Int64
model                      object
Hand                        Int64
Gear                     category
capacity_Engine             Int64
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area               string[python]
City               string[python]
Price                     float64
Pic_num                     Int64
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description        string[python]
Color              string[python]
Km                          Int64
Test                        Int64
dtype: object

In [82]:
filtered_data.head(10)

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test
0,קיה,2011,פורטה,2,אוטומטית,1600,בנזין,השכרה,פרטית,מושבים בשפלה,פתחיה,16000.0,5,2024-05-26,2024-06-07,"לרציניים בלבד .קיה פורטה, הרכב היום בבעלות פרט...",כסוף מטאלי,227000,200
1,קיה,2016,פיקנטו,1,אוטומטית,1200,בנזין,פרטית,פרטית,רמת גן - גבעתיים,בני ברק,28500.0,3,2024-05-31,2024-05-31,מצב מצוין לא עברה שום תיקון צבע או תאונה כל שהיא,אפור,170000,259
2,קיה,2014,פיקנטו,3,ידנית,1000,בנזין,פרטית,פרטית,עמק יזרעאל,גן נר,15000.0,1,2024-04-21,2024-05-18,קטנה שמורה עם טסט לשנה,ורוד,135000,290
4,קיה,2012,ריו,4,ידנית,1400,דיזל,פרטית,פרטית,אשדוד - אשקלון,אשדוד,18000.0,6,2024-05-07,2024-05-07,רכב חסכוני בטירוף אחד ל 22 טסט לשנה אחרי טיפול...,לבן שנהב,207000,320
7,קיה,2014,פיקנטו,3,ידנית,1000,בנזין,פרטית,פרטית,עמק יזרעאל,גן נר,17000.0,3,2024-04-02,2024-04-02,"קיה מתוקה ושמורה, ידנית! בצבע ורוד! נמכרת עקב ...",ורוד,135000,290
8,קיה,2014,פיקנטו,2,אוטומטית,1200,בנזין,פרטית,פרטית,רמלה - לוד,רמלה,37000.0,5,2024-03-27,2024-03-27,רכב שמור מאוד!! טסט עד 2/2025 תוספות מוליטימדי...,אפור מטאלי,92000,259
11,קיה,2014,פיקנטו,4,אוטומטית,1200,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,30500.0,3,2024-02-29,2024-03-04,קילומטראז 129 אלף השנה הוחלף מצבר ורצועת מנוע ...,לבן,129000,108
13,קיה,2014,פיקנטו,3,אוטומטית,1200,בנזין,פרטית,פרטית,גליל ועמקים,נצרת עילית,35000.0,4,2024-02-25,2024-02-25,קיה פיקנטו ספורט מנוע 1250 עם 84 כס יד שלישית ...,לבן,143000,<NA>
14,קיה,2014,פורטה,3,אוטומטית,1600,בנזין,השכרה,פרטית,נס ציונה - רחובות,רחובות,24000.0,9,2024-02-10,2024-02-10,רכב שמור מאוד נמכר עקב קבלת רכב מהעבודה,לבן,180174,<NA>
16,קיה,2015,פורטה,2,אוטומטית,1600,בנזין,אחר,פרטית,באר שבע והסביבה,אופקים,36000.0,5,2024-01-11,2024-01-12,"רכב שמור, מטופל בזמן, אמין! תוספות מובילאיי, מ...",אפור,141000,<NA>


### API 

Fetching and filtering car data from Israeli government API

In [83]:
import requests
import json

In [84]:
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6'
web = requests.get(url)
dict_data = json.loads(web.text)

In [85]:
api_list = []
for number in range(0,len(dict_data["result"]["records"])):
    tozar = dict_data["result"]["records"][number]["tozar"]
    degem = dict_data["result"]["records"][number]["kinuy_mishari"]
    shnaton = dict_data["result"]["records"][number]["shnat_yitzur"]
    dictio = {'tozar':tozar,'degem':degem,'shnaton':shnaton}
    api_list.append(dictio)

In [86]:
car_gov_data = pd.DataFrame(api_list)
kia = car_gov_data.loc[car_gov_data['tozar']=='קיה']

Standardizing and Filtering Kia Car Model Names

In [87]:
translated_name = {
    'CARENS': 'קארנס',
    'CARNIVAL': 'קרניבל',
    'CEED': 'סיד',
    'CERAT0': ['קריאטו', 'סראטו'],
    'CERATO': ['קריאטו', 'סראטו'],
    'PICANT0': 'פיקנטו',
    'PICANTO': 'פיקנטו',
    'PIKANTO': 'פיקנטו',
    'FORTE': 'פורטה',
    'RI0': 'ריו',
    'RIO': 'ריו',
    'NIRO': 'נירו',
    'OPTIMA': 'קיה אופטימה / Optima',
    'SOUL': 'סול',
    'SHUMA': 'שומה',
    'JOICE': "ג'ויס",
    'PRIDE': 'פרייד'
}

In [88]:
def translate_degem(degem):
    translation = translated_name.get(degem, degem) 
    if isinstance(translation, list):
        return '/'.join(translation)  
    return translation

In [89]:
kia['degem_translated'] = kia['degem'].apply(translate_degem)

In [90]:
translated_values = list(translated_name.values())
translated_values = [item for sublist in translated_values for item in (sublist if isinstance(sublist, list) else [sublist])]

In [91]:
kia_filtered = kia[kia['degem_translated'].isin(translated_values)]
kia_filtered

,tozar,degem,shnaton,degem_translated
15,קיה,CEED,2016,סיד
52,קיה,CARNIVAL,2009,קרניבל
69,קיה,RIO,2015,ריו
114,קיה,FORTE,2016,פורטה
166,קיה,CEED,2016,סיד
...,...,...,...,...
31862,קיה,RI0,2012,ריו
31943,קיה,PRIDE,1998,פרייד
31965,קיה,SOUL,2017,סול
31977,קיה,CARENS,2003,קארנס


Counting and renaming supply of standardized Kia car models

In [92]:
identical_rows_count = kia_filtered.groupby(['tozar', 'shnaton', 'degem_translated']).size().reset_index(name='count')
identical_rows_count = identical_rows_count.rename(columns={'tozar': 'manufactor', 'shnaton': 'Year', 'degem_translated': 'model', 'count':'supply'})
identical_rows_count

,manufactor,Year,model,supply
0,קיה,1996,פרייד,2
1,קיה,1997,פרייד,2
2,קיה,1998,פרייד,2
3,קיה,1999,פרייד,1
4,קיה,1999,קרניבל,1
...,...,...,...,...
133,קיה,2023,סיד,2
134,קיה,2023,פיקנטו,4
135,קיה,2023,קרניבל,2
136,קיה,2023,ריו,2


Assigning supply scores to filtered Kia car dataset

In [93]:
count_dict = {(row['manufactor'], row['Year'], row['model']): row['supply'] for idx, row in identical_rows_count.iterrows()}

def get_supply(row):
    return count_dict.get((row['manufactor'], row['Year'], row['model']), 0)
    
filtered_data['Supply_score'] = filtered_data.apply(get_supply, axis=1)

In [96]:
filtered_data.head(10)

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,קיה,2011,פורטה,2,אוטומטית,1600,בנזין,השכרה,פרטית,מושבים בשפלה,פתחיה,16000.0,5,2024-05-26,2024-06-07,"לרציניים בלבד .קיה פורטה, הרכב היום בבעלות פרט...",כסוף מטאלי,227000,200,4
1,קיה,2016,פיקנטו,1,אוטומטית,1200,בנזין,פרטית,פרטית,רמת גן - גבעתיים,בני ברק,28500.0,3,2024-05-31,2024-05-31,מצב מצוין לא עברה שום תיקון צבע או תאונה כל שהיא,אפור,170000,259,24
2,קיה,2014,פיקנטו,3,ידנית,1000,בנזין,פרטית,פרטית,עמק יזרעאל,גן נר,15000.0,1,2024-04-21,2024-05-18,קטנה שמורה עם טסט לשנה,ורוד,135000,290,14
4,קיה,2012,ריו,4,ידנית,1400,דיזל,פרטית,פרטית,אשדוד - אשקלון,אשדוד,18000.0,6,2024-05-07,2024-05-07,רכב חסכוני בטירוף אחד ל 22 טסט לשנה אחרי טיפול...,לבן שנהב,207000,320,9
7,קיה,2014,פיקנטו,3,ידנית,1000,בנזין,פרטית,פרטית,עמק יזרעאל,גן נר,17000.0,3,2024-04-02,2024-04-02,"קיה מתוקה ושמורה, ידנית! בצבע ורוד! נמכרת עקב ...",ורוד,135000,290,14
8,קיה,2014,פיקנטו,2,אוטומטית,1200,בנזין,פרטית,פרטית,רמלה - לוד,רמלה,37000.0,5,2024-03-27,2024-03-27,רכב שמור מאוד!! טסט עד 2/2025 תוספות מוליטימדי...,אפור מטאלי,92000,259,14
11,קיה,2014,פיקנטו,4,אוטומטית,1200,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,30500.0,3,2024-02-29,2024-03-04,קילומטראז 129 אלף השנה הוחלף מצבר ורצועת מנוע ...,לבן,129000,108,14
13,קיה,2014,פיקנטו,3,אוטומטית,1200,בנזין,פרטית,פרטית,גליל ועמקים,נצרת עילית,35000.0,4,2024-02-25,2024-02-25,קיה פיקנטו ספורט מנוע 1250 עם 84 כס יד שלישית ...,לבן,143000,<NA>,14
14,קיה,2014,פורטה,3,אוטומטית,1600,בנזין,השכרה,פרטית,נס ציונה - רחובות,רחובות,24000.0,9,2024-02-10,2024-02-10,רכב שמור מאוד נמכר עקב קבלת רכב מהעבודה,לבן,180174,<NA>,9
16,קיה,2015,פורטה,2,אוטומטית,1600,בנזין,אחר,פרטית,באר שבע והסביבה,אופקים,36000.0,5,2024-01-11,2024-01-12,"רכב שמור, מטופל בזמן, אמין! תוספות מובילאיי, מ...",אפור,141000,<NA>,6


Saving the file to a CSV file on a PC

In [95]:
filtered_data.to_csv('Final_part_1.csv',index=False ,encoding = 'utf-8-sig')